In [6]:

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import yfinance as yf
import pandas_ta as ta
from transformers import pipeline
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import Dataset, DataLoader

# Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SEQ_LEN = 60  # Input sequence length (1 hour)
PREDICTION_HORIZON_24H = 24
PREDICTION_HORIZON_1WEEK = 168
BATCH_SIZE = 32

# 1. Data Loading & Preprocessing
def fetch_data():
    # Fetch Bitcoin historical data
    btc = yf.download('BTC-USD', start='2023-03-25', end='2025-01-24', interval='1h')

    # Calculate Technical Indicators:cite[1]:cite[2]
    btc['RSI'] = ta.rsi(btc.Close, length=14)
    btc['MACD'] = ta.macd(btc.Close, fast=12, slow=26, signal=9)['MACD_12_26_9']
    btc['VWAP'] = ta.vwap(btc.High, btc.Low, btc.Close, btc.Volume)
    btc = btc.dropna()

    # Simulated Sentiment Data (Replace with real API calls):cite[7]:cite[8]
    sentiment_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
    sample_texts = ["Bitcoin will skyrocket!", "Market crash incoming"] * 1000  # Mock data
    sentiments = [sentiment_model(text[:512])[0]['score'] for text in sample_texts[:len(btc)]]
    btc['Sentiment'] = sentiments[:len(btc)]  # Align length

    return btc

# 2. Dataset Preparation
class BitcoinDataset(Dataset):
    def __init__(self, data, seq_len, pred_horizon):
        self.scaler = MinMaxScaler(feature_range=(-1, 1))
        self.data = self.scaler.fit_transform(data[['Close', 'RSI', 'MACD', 'VWAP', 'Sentiment']])
        self.seq_len = seq_len
        self.pred_horizon = pred_horizon

    def __len__(self):
        return len(self.data) - self.seq_len - self.pred_horizon

    def __getitem__(self, idx):
        x = self.data[idx:idx+self.seq_len]
        y = self.data[idx+self.seq_len:idx+self.seq_len+self.pred_horizon, 0]  # Predict Close price
        return torch.FloatTensor(x).to(device), torch.FloatTensor(y).to(device)

# 3. Transformer Model Architecture
class BitcoinTransformer(nn.Module):
    def __init__(self, input_dim=5, d_model=128, nhead=4, num_layers=4, dropout=0.1):
        super().__init__()
        self.embedding = nn.Linear(input_dim, d_model)
        self.pos_encoder = nn.TransformerEncoderLayer(d_model, nhead, dropout=dropout)
        self.transformer = nn.TransformerEncoder(self.pos_encoder, num_layers=num_layers)
        self.decoder = nn.Linear(d_model, 1)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(1, 0, 2)  # (seq_len, batch, d_model)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # (batch, seq_len, d_model)
        x = self.decoder(x[:, -1, :])  # Use last timestep for prediction
        return x

# 4. Training Loop
def train_model(dataset, model, epochs=50):
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    model.train()
    for epoch in range(epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

# 5. Prediction & Visualization
def predict_future(model, dataset, steps):
    model.eval()
    with torch.no_grad():
        x = dataset[-SEQ_LEN:][None, ...]  # Last sequence
        preds = []
        for _ in range(steps):
            pred = model(torch.FloatTensor(x).to(device))
            preds.append(pred.cpu().numpy())
            x = np.roll(x, -1, axis=1)
            x[:, -1, 0] = pred.item()  # Update Close price
        return dataset.scaler.inverse_transform(np.array(preds).reshape(-1, 1))

# Main Execution
if __name__ == "__main__":
    # Load data
    data = fetch_data()
    dataset_24h = BitcoinDataset(data, SEQ_LEN, PREDICTION_HORIZON_24H)
    dataset_1week = BitcoinDataset(data, SEQ_LEN, PREDICTION_HORIZON_1WEEK)

    # Initialize model
    model = BitcoinTransformer().to(device)

    # Train for 24h prediction
    train_model(dataset_24h, model)
    pred_24h = predict_future(model, dataset_24h, PREDICTION_HORIZON_24H)

    # Train for 1-week prediction (adjust hyperparameters for longer horizons)
    train_model(dataset_1week, model)
    pred_1week = predict_future(model, dataset_1week, PREDICTION_HORIZON_1WEEK)

    # Plot results
    import matplotlib.pyplot as plt
    plt.figure(figsize=(12, 6))
    plt.plot(data['Close'].values[-500:], label='Historical Price')
    plt.plot(np.arange(500, 500+PREDICTION_HORIZON_24H), pred_24h, label='24h Prediction')
    plt.plot(np.arange(500, 500+PREDICTION_HORIZON_1WEEK), pred_1week, label='1-Week Prediction')
    plt.legend()
    plt.show()

[*********************100%***********************]  1 of 1 completed


TypeError: 'NoneType' object is not subscriptable